ref: https://www.kaggle.com/hamishdickson/bidirectional-lstm-in-keras-with-glove-embeddings

ref: https://lovit.github.io/nlp/representation/2018/09/05/glove/

In [1]:
import pandas as pd
import time
import os, gc
import numpy as np
from tqdm import tqdm
import random

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import *
from keras.utils.np_utils import to_categorical
from keras.initializers import Constant
import re

import matplotlib.pyplot as plt
%matplotlib inline

import torch

Using TensorFlow backend.


In [2]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(42)

In [3]:
%%time
#train_df=pd.read_csv("../KB_NLP/morphs/komo_morphs_train.csv")
#test_df=pd.read_csv("../KB_NLP/morphs/komo_morphs_test.csv")
train_df=pd.read_csv("../KB_NLP/raw_data/train.csv")
test_df=pd.read_csv("../KB_NLP/raw_data/public_test.csv")

CPU times: user 906 ms, sys: 61.3 ms, total: 968 ms
Wall time: 967 ms


In [4]:
pd.set_option('display.max_colwidth',-1)
train_df.head()

,id,year_month,text,smishing
0,0,2017-01,XXX은행성산XXX팀장입니다.행복한주말되세요,0
1,1,2017-01,오늘도많이웃으시는하루시작하세요XXX은행 진월동VIP라운지 XXX올림,0
2,2,2017-01,안녕하십니까 고객님. XXX은행입니다.금일 납부하셔야 할 금액은 153600원 입니다.감사합니다. 새해 복 많이 받으십시오.XXX은행옥포XXX올림,0
3,4,2017-01,XXX 고객님안녕하세요XXX은행 XXX지점입니다지난 한 해 동안 저희 XXX지점에 보내주신 성원에 감사드립니다. 설렘으로 시작한 2017년소망하시는 일 모두 이XXX 고객님의 가정에 늘 건강과 행복이 함께하길 기원하겠습니다. 사랑하는 가족과 함께 정을 나누는 행복한 설 명절 보내세요 XXX은행 XXX지점직원일동,0
4,5,2017-01,1월은 새로움이 가득XXX입니다.올 한해 더 많이행복한 한해되시길바랍니다,0


In [5]:
from soynlp.hangle import decompose

doublespace_pattern = re.compile('\s+')

def jamo_sentence(sent):

    def transform(char):
        if char == ' ':
            return char
        cjj = decompose(char)
        try:
            len(cjj)
        except:
            return ' '
        if len(cjj) == 1:
            return cjj
        cjj_ = ''.join(c if c != ' ' else '' for c in cjj)
        return cjj_

    sent_ = ''.join(transform(char) for char in sent)
    sent_ = doublespace_pattern.sub(' ', sent_)
    return sent_
# 'ㅇㅓ-ㅇㅣ-ㄱㅗ- ㅋㅔㄱㅋㅔㄱ ㅇㅏ-ㅇㅣ-ㄱㅗ-ㅇㅗ-'

jamo_sentence(train_df.loc[0, 'text'])

' ㅇㅡㄴㅎㅐㅇㅅㅓㅇㅅㅏㄴ ㅌㅣㅁㅈㅏㅇㅇㅣㅂㄴㅣㄷㅏ ㅎㅐㅇㅂㅗㄱㅎㅏㄴㅈㅜㅁㅏㄹㄷㅚㅅㅔㅇㅛ'

In [6]:
%%time
test_df['jamo'] = test_df['text'].apply(lambda x: jamo_sentence(x))
train_df['jamo'] = train_df['text'].apply(lambda x: jamo_sentence(x))

CPU times: user 1min, sys: 276 ms, total: 1min
Wall time: 1min


In [7]:
%%time
train_df.to_csv("../KB_NLP/jamo_data/jamo_train.csv", index=False)
test_df.to_csv("../KB_NLP/jamo_data/jamo_test.csv", index=False)

CPU times: user 2.83 s, sys: 196 ms, total: 3.02 s
Wall time: 5.74 s


In [8]:
raw_corpus_fname = '\n'.join(pd.concat([train_df['jamo'], test_df['jamo']]))
file=open('../KB_NLP/jamo_text.txt','w')
file.write(raw_corpus_fname)
file.close()

In [9]:
jamo_jumo = pd.concat([train_df['jamo'], test_df['jamo']])

In [10]:
input_sentences = list(jamo_jumo)

### window 1

In [11]:
from soynlp.utils import DoublespaceLineCorpus
from soynlp.vectorizer import sent_to_word_contexts_matrix

#corpus_path = '2016-10-20_article_all_normed_ltokenize.txt'
#corpus = DoublespaceLineCorpus(corpus_path, iter_sent=True)

x, idx2vocab = sent_to_word_contexts_matrix(
    input_sentences,
    windows=1,
    min_tf=10,
    tokenizer=lambda x:x.split(), # (default) lambda x:x.split(),
    dynamic_weight=True,
    verbose=True)

print(x.shape) # (36002, 36002)

Create (word, contexts) matrix
  - counting word frequency from 297570 sents, mem=1.015 Gb
  - scanning (word, context) pairs from 297570 sents, mem=1.133 Gb
  - (word, context) matrix was constructed. shape = (38890, 38890)                    
  - done
(38890, 38890)


In [12]:
from glove import Corpus, Glove

In [13]:
glove = Glove(no_components=200, learning_rate=0.01, random_state=42)
glove.fit(x.tocoo(), epochs=5, no_threads=4, verbose=True)

Performing 5 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4


In [14]:
dictionary = {vocab:idx for idx, vocab in enumerate(idx2vocab)}
glove.add_dictionary(dictionary)

In [15]:
glove_embedding = pd.DataFrame()
glove_embedding['key'] = list(glove.dictionary.keys())
glove_embedding['vectors'] = list(glove.word_vectors)
glove_embedding['vectors'] = glove_embedding['vectors'].map(str)
glove_embedding['vectors'] = glove_embedding['key'] + glove_embedding['vectors']
for i in tqdm(range(len(glove_embedding))):
    glove_embedding['vectors'][i] = glove_embedding['vectors'][i].replace('[','').replace(']','').replace('\n','')
glove_embedding.head()

100%|██████████| 38890/38890 [00:07<00:00, 5092.39it/s]


,key,vectors
0,ㅇㅗㄹㄹㅣㅁ,ㅇㅗㄹㄹㅣㅁ-0.26324613 -0.02368496 0.17138936 0.31919709 -0.00689488 -0.44605252 -0.48537465 0.31311969 -0.27306892 -0.3357058 -0.45830763 0.43211633 0.47088084 -0.39937638 0.08947951 -0.40416604 -0.45353923 0.32876612 0.36043311 0.19995081 0.38421529 -0.39928572 0.15184268 0.01312085 -0.06739303 0.3669841 -0.3326004 0.20394546 -0.3291216 -0.30769685 -0.19931577 -0.2018717 -0.39794443 0.18234525 0.34545889 0.22885314 0.14080159 -0.42226852 0.44131197 -0.27124807 0.26018056 0.48561258 -0.44012502 0.40871661 -0.21198955 0.32536252 0.20979982 0.40596122 -0.2680429 -0.35655377 -0.16324139 0.35765051 0.63067254 0.33770794 -0.40297928 0.40558892 -0.42941914 -0.42061108 -0.32547986 -0.37210191 -0.35845763 0.32935819 0.0761545 -0.31737562 0.3506474 -0.25521387 -0.3401427 -0.30469031 -0.47357609 0.28129283 0.25701576 -0.38337743 0.20355264 0.35353469 0.35608287 -0.18183028 0.58787545 -0.11258498 0.1264746 -0.44819501 0.30213598 0.23646804 -0.34526207 -0.41680279 -0.32883283 -0.48248721 0.26442236 -0.07552682 0.36177159 0.10534863 -0.38082651 -0.08936218 0.32863151 -0.4269309 -0.28959362 0.2240692 -0.23371203 -0.4738351 -0.31996831 -0.43013864 -0.45742153 0.49453867 0.10599406 0.22118925 0.39092379 -0.45031061 -0.52173794 0.35475314 0.32576598 -0.14111253 0.30286463 -0.43283955 0.32750013 0.36729262 0.47768621 0.39781853 -0.1007444 0.09847619 0.44335116 -0.34536784 0.29290384 0.44129907 0.2028668 -0.24157026 -0.02368141 0.30631031 0.20524209 -0.1190077 -0.52472569 0.25885506 -0.3380466 0.37261 -0.42373552 0.28293189 -0.15180832 -0.02618632 -0.21165631 0.43343501 -0.22762538 -0.13681001 0.48857164 -0.3827457 0.40211896 0.26774224 -0.24846846 -0.32445273 0.5817532 -0.43216049 -0.36291881 0.47065761 0.37897837 0.25206909 -0.11672433 0.11441597 -0.22376758 0.16660629 0.64783438 0.30849193 0.09992503 0.34579599 -0.23138357 -0.23267252 -0.31017169 -0.10552328 -0.47186511 0.37932072 0.3544638 -0.3718918 -0.47914602 -0.20357382 -0.07177603 -0.08657376 -0.37860146 -0.28813909 -0.22686767 -0.38728035 0.10554038 -0.31160923 0.36114391 -0.29160718 -0.42616864 -0.03577871 -0.21910479 0.50787824 -0.12274066 0.27736672 0.4060941 -0.11171136 0.16975818 0.24908089 -0.43790707 0.61148535 0.49996092 0.32663876 -0.32662898 0.2597609 0.36299223 0.37074482 0.4155068 0.35441925
1,ㅇㅡㄴㅎㅐㅇ,ㅇㅡㄴㅎㅐㅇ-0.17145332 -0.20260989 0.11733715 0.28826444 0.21289365 -0.27726599 -0.29386979 0.30689655 -0.35514082 -0.3096952 -0.21365526 0.25867431 0.10199709 -0.37072184 0.15710515 -0.28837153 -0.27488793 0.22558779 0.23741683 0.15106138 0.12315886 -0.32122207 0.03742321 0.11340612 -0.24253265 0.21370225 -0.18026904 0.09764091 -0.1209306 -0.17731586 -0.06910689 -0.02806085 -0.1393988 0.12302363 -0.09050706 0.16008655 -0.17265485 -0.31220133 0.17969053 -0.14110015 0.21758027 0.10329363 -0.12288101 0.12494619 -0.17842359 0.32303197 0.0910653 0.28906552 0.0307323 -0.23746819 -0.27590243 0.23001134 0.22142613 0.28695002 0.12641401 0.25293844 -0.05722432 -0.33271881 -0.12802003 -0.24565919 -0.17858802 0.2689243 0.06282523 -0.16481707 0.36932831 -0.22648502 -0.31342403 -0.18730119 -0.13730998 -0.10937504 -0.12669457 0.02423994 0.23199514 0.27702759 0.27204582 0.00619051 0.05907741 0.11715736 0.0679795 -0.10779382 0.09219567 0.21890166 -0.25774489 -0.35416836 -0.20610571 -0.17342196 0.18165003 -0.1177088 0.19645612 0.12174515 -0.29548975 -0.1573005 0.2309999 -0.04933441 -0.22904556 0.19482473 -0.06454602 -0.28815669 -0.10041897 -0.32849347 -0.20360489 0.03762246 -0.04481123 0.1481647 0.24682075 -0.07884117 -0.09849821 0.05206405 0.26368936 -0.09566898 0.23993819 -0.21160782 0.15622702 0.22002858 0.19215324 0.21632498 -0.21706278 0.25877331 0.12869082 -0.18418742 0.11543104 0.19500451 0.13231774 -0.2090333 0.17654631 0.27724033 -0.13496149 -0.21652007 -0.06255342 0.1992889 -0.18175081 0.24989826 -0.29987899 -0.12544393 -0.30784739 -0.22598934 -0.1710232 0.14685071 -0.18917296 -0.25185269 0.25686527 -0.2184126 0.17525151 0.26052604 -0.16317637 -0.17620178 0.19003763 -0.18

### window 3

In [16]:
from soynlp.utils import DoublespaceLineCorpus
from soynlp.vectorizer import sent_to_word_contexts_matrix

#corpus_path = '2016-10-20_article_all_normed_ltokenize.txt'
#corpus = DoublespaceLineCorpus(corpus_path, iter_sent=True)

x, idx2vocab = sent_to_word_contexts_matrix(
    input_sentences,
    windows=3,
    min_tf=10,
    tokenizer=lambda x:x.split(), # (default) lambda x:x.split(),
    dynamic_weight=True,
    verbose=True)

print(x.shape) # (36002, 36002)

Create (word, contexts) matrix
  - counting word frequency from 297570 sents, mem=1.432 Gb
  - scanning (word, context) pairs from 297570 sents, mem=1.732 Gb
  - (word, context) matrix was constructed. shape = (38890, 38890)                    
  - done
(38890, 38890)


In [17]:
from glove import Corpus, Glove

In [18]:
glove = Glove(no_components=200, learning_rate=0.01, random_state=42)
glove.fit(x.tocoo(), epochs=5, no_threads=4, verbose=True)

Performing 5 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4


In [19]:
dictionary = {vocab:idx for idx, vocab in enumerate(idx2vocab)}
glove.add_dictionary(dictionary)

In [20]:
glove_embedding = pd.DataFrame()
glove_embedding['key'] = list(glove.dictionary.keys())
glove_embedding['vectors'] = list(glove.word_vectors)
glove_embedding['vectors'] = glove_embedding['vectors'].map(str)
glove_embedding['vectors'] = glove_embedding['key'] + glove_embedding['vectors']
for i in tqdm(range(len(glove_embedding))):
    glove_embedding['vectors'][i] = glove_embedding['vectors'][i].replace('[','').replace(']','').replace('\n','')
glove_embedding.head()

100%|██████████| 38890/38890 [00:07<00:00, 5517.08it/s]


,key,vectors
0,ㅇㅗㄹㄹㅣㅁ,ㅇㅗㄹㄹㅣㅁ-0.17762808 -0.18490076 0.08588548 0.24284263 -0.16239751 -0.27436715 -0.74741488 0.27707478 -0.25259406 -0.23154107 -0.33708273 0.22697088 0.64465582 -0.33283018 0.22061267 -0.00110232 -0.32557432 -0.18232905 0.23525859 -0.09731015 0.23999817 -0.32069043 0.0236779 -0.28442625 -0.21153094 0.27907981 -0.1574479 0.07752668 0.00385965 -0.21277331 -0.12128502 -0.13509534 -0.26911675 -0.49661489 -0.09681506 0.4846365 0.2365633 -0.19005776 0.65949728 0.14886284 0.19654134 0.15389083 -0.45425171 0.25612321 0.14431148 0.22433817 0.24232983 0.17212922 -0.51305458 -0.16731921 -0.24546143 -0.0804945 0.06065588 0.20904637 -0.19280202 0.24300949 0.28010348 -0.30338969 -0.03123301 0.07157973 -0.05354185 0.21945012 0.11181493 -0.15549045 0.25489995 -0.21970318 -0.27870176 -0.20826254 -0.01117263 0.27930547 0.17158264 0.14082292 0.13147954 0.1849325 0.29558346 0.56883546 -0.0227893 0.08927379 0.09878307 -0.37594125 0.26457799 0.14444631 -0.28795017 -0.26844787 -0.21988838 -0.21435549 0.06801286 -0.01953159 0.29221798 -0.11215994 -0.28065159 -0.12377167 0.26491115 0.09103644 -0.21920181 0.30420816 0.20697315 -0.31834055 0.05210666 -0.24388458 -0.26418875 0.23847078 -0.11318378 -0.18057553 0.21161032 -0.34224401 0.09544777 -0.03980369 0.24241325 -0.2171742 0.20423383 -0.76855712 0.19273149 0.26920546 -0.01427855 0.39934365 -0.10466226 0.03485697 0.51745231 0.21565218 0.27542995 0.18734969 0.23824979 0.10376377 0.15228127 0.22111061 -0.11053208 -0.20144308 -0.33583225 0.24846504 -0.12304139 0.2279901 -0.31229923 -0.39185089 -0.01723182 -0.43611997 0.50799385 0.2643226 -0.20321429 -0.16495067 0.27555359 -0.22156065 0.24016995 0.22833714 -0.1071268 -0.08148818 0.19848486 -0.22435449 -0.1045391 0.27179609 0.09831652 0.25171847 0.04511269 0.1662873 -0.18367904 0.21315035 0.78707128 0.27195917 0.17559253 0.24040041 -0.15316599 -0.23114456 -0.23123284 -0.16216958 -0.33669661 0.25399899 0.27594369 -0.2417251 -0.25129025 -0.13803024 -0.05740072 0.20420629 -0.29767144 -0.75723891 -0.04349043 -0.24188759 0.17401016 -0.15281127 0.13742357 -0.14885555 -0.26885002 0.06463839 0.56806788 0.18415101 0.05845328 0.2345564 0.7332153 0.22757088 0.20190615 0.16595557 -0.21651237 0.80447485 0.1661763 -0.03948209 -0.28126886 0.19478424 0.22551864 0.09433982 0.41178246 0.22315449
1,ㅇㅡㄴㅎㅐㅇ,ㅇㅡㄴㅎㅐㅇ-0.16453087 -0.26516197 0.04300108 0.21623661 0.32732233 -0.18126474 -0.56323844 0.28043292 -0.31980237 -0.25807859 -0.29012107 0.15741141 0.31911365 -0.34843289 0.26862883 -0.10760345 -0.30689119 -0.15652791 0.2239783 0.00326467 0.14461368 -0.32033049 -0.20757623 -0.05716952 -0.2671825 0.23523486 -0.15048179 0.09631036 -0.00559792 -0.23238276 -0.04263311 0.13703183 -0.14520505 -0.25156698 -0.19270911 0.48297001 0.07315754 -0.15967866 0.51416661 0.13045074 0.25031876 0.11132543 0.04368249 0.23119788 0.01156637 0.2950039 0.16231588 0.18430469 -0.10211864 -0.16909193 -0.28898996 -0.04575224 -0.07276394 0.23463 0.06735839 0.21228657 0.51122365 -0.31910732 0.28320037 0.02553759 -0.05598385 0.2634646 0.14276917 -0.17729745 0.31753387 -0.2573959 -0.30861766 -0.15715178 0.08164875 0.08651492 -0.03669455 0.16815502 0.19832007 0.15225004 0.3014319 0.38917697 -0.12903879 0.06920402 0.04116963 0.13911576 -0.01487776 0.16264057 -0.25780852 -0.25753512 -0.20985902 -0.18992802 0.0622322 -0.06275048 0.23737632 -0.06381908 -0.2807851 -0.2059566 0.25786982 0.15717269 -0.19918899 0.51231644 0.07148164 -0.29569274 0.10124513 -0.25240034 -0.23518909 0.11731936 -0.16521177 0.41369881 0.15569699 -0.09872513 0.17704693 -0.03653351 0.26531369 -0.24031066 0.14539152 -0.47667134 0.20120067 0.26080958 0.00652634 0.25945315 -0.18929608 0.14036646 0.26666941 0.21742156 0.24451342 0.11021287 0.25799339 0.11037641 0.24183836 0.2304094 -0.19287689 -0.26653333 -0.18277262 0.22896443 -0.13014098 0.21826324 -0.25475997 -0.17116112 -0.15806132 -0.62726317 0.42995061 0.21922358 -0.25783081 -0.23042489 0.27291364 -0.2224107 0.21040032 0.26385077 -0.10036656 -0.05226875 0.11469902 -0.17613

In [16]:
%%time
glove = Glove(no_components=200, learning_rate=0.01, random_state=42)
glove.fit(x.tocoo(), epochs=100, no_threads=4, verbose=True)

Performing 100 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Epoch 90
Epoch 91
Epoch 92
Epoch 93
Epoch 94
Epoch 95
Epoch 96
Epoch 97
Epoch 98
Epoch 99
CPU times: user 5min 2s, sys: 47 ms, total: 5min 2s
Wall time: 1

In [17]:
dictionary = {vocab:idx for idx, vocab in enumerate(idx2vocab)}
glove.add_dictionary(dictionary)

In [18]:
glove.word_vectors.shape

(38890, 200)

In [19]:
glove.word_vectors[:1]

array([[-0.04014644,  0.14591996,  0.21770952,  0.27505619, -0.21774962,
        -0.3299259 , -0.34875664,  0.15955163, -0.30599767, -0.09654434,
        -0.36265585,  0.36642118,  0.58957813, -0.25477741, -0.77548881,
        -0.16812614, -0.30386485,  0.3014517 ,  0.11541772,  0.23872045,
         0.23607387, -0.32231933,  0.65029271, -0.57035232,  0.3518528 ,
         0.2721458 ,  0.29503168, -0.24414353,  0.24229787, -0.27020786,
         0.02250393, -0.64149403, -0.18691672,  0.57878591,  0.04031945,
         0.25008333, -0.11378724, -0.24900752, -0.36675543, -0.16945304,
        -0.26521683,  0.47874288, -0.39373579,  0.35531035, -0.0364649 ,
         0.24267091,  0.28361149,  0.0727594 , -0.18305883,  0.18589462,
         0.01891854,  0.31339466,  0.2110837 ,  0.2616362 ,  0.1850178 ,
         0.30698852, -0.34319437, -0.33101811, -0.0366438 , -0.10888972,
        -0.33897783, -0.18377894,  0.22498183, -0.16396334,  0.17689501,
        -0.09917777, -0.23181384, -0.07816078, -0.3

In [21]:
glove_embedding = pd.DataFrame()
glove_embedding['key'] = list(glove.dictionary.keys())
glove_embedding['vectors'] = list(glove.word_vectors)

,key,vectors
0,ㅇㅗㄹㄹㅣㅁ,"[-0.040146439146532854, 0.1459199564025886, 0.21770951649967288, 0.27505618905214874, -0.21774961539982576, -0.32992590369903885, -0.34875663718111205, 0.15955162607922904, -0.305997672446965, -0.09654433939962898, -0.3626558497403411, 0.3664211784936428, 0.5895781267518013, -0.25477740553566475, -0.7754888061202576, -0.16812614305628376, -0.3038648472127392, 0.3014516981329455, 0.11541771856676092, 0.23872045098218483, 0.23607387493931112, -0.3223193297401064, 0.6502927086724973, -0.5703523150599951, 0.3518527951391747, 0.2721457953773931, 0.2950316778258804, -0.24414352851925353, 0.2422978729500584, -0.27020786486751874, 0.022503931170752046, -0.6414940339397746, -0.18691671529030046, 0.5787859053707188, 0.04031944855763691, 0.25008332771133435, -0.11378724404866228, -0.2490075170225759, -0.3667554252195786, -0.16945303784637283, -0.2652168322474747, 0.47874287732138326, -0.3937357934693531, 0.3553103520420614, -0.03646489812035994, 0.2426709065582876, 0.283611489751039, 0.07275939930438592, -0.18305883253701133, 0.18589461868555524, 0.018918544221754655, 0.3133946598397176, 0.2110837041232433, 0.2616362013122453, 0.1850177996830624, 0.3069885215742826, -0.3431943654946833, -0.33101811024047656, -0.036643801577073046, -0.10888971595732212, -0.3389778346535938, -0.18377894383819907, 0.22498182770753475, -0.16396333881447284, 0.17689500936933583, -0.09917777024020573, -0.2318138382632774, -0.07816078123507537, -0.32654060854510836, 0.3917461140143674, 0.09102456177488191, 0.0580177063672472, -0.5810236975096904, -0.15678636539390664, 0.12635572264744868, 0.003218629273790827, 0.0629406559152243, -0.3518278754445992, -0.04854144943131424, -0.12647429335202598, 0.3510986852746356, 0.4051464808002084, -0.21645253747254242, -0.19755681746718312, -0.3228944423688359, -0.3002770816990503, 0.3483795838100865, -0.046157589155479815, 0.3533725154836427, -0.06437188334279607, -0.30470854975854267, 0.7943502240375381, 0.25375599377636954, -0.3947189615781569, -0.04646276048762151, 0.28683884154846023, 0.5676365611630606, -0.3430480468733849, -0.13901352788172927, -0.2517575370388552, ...]"
1,ㅇㅡㄴㅎㅐㅇ,"[-0.15727920467170306, -0.08054372063028327, -0.2784167129656523, 0.21973003896583776, 0.1345708537280672, -0.2774422316098419, -0.400841878504431, 0.30184944983642836, -0.37914002232502925, -0.2627290278595458, -0.16365064306242355, 0.1128674486396726, 0.3849630509308168, -0.3393446132997946, -0.3736597817841855, -0.29523862168727905, -0.30348419564073675, -0.08834723476573174, -0.07348613400918769, 0.10211750359834441, 0.12998961298497563, -0.3081142786792763, -0.3924729227360429, 0.01906074899715463, -0.07931723237395684, 0.05273993815782015, 0.6745660958607178, -0.5018620313279732, 0.774314659482358, -0.23300255532277203, -0.07766038389033927, 0.1186569755027705, -0.09071159349706298, 0.12439919776911805, -0.03220682619672916, -0.3979169872948048, -0.1483416568066553, -0.3256488848834462, 0.3223875163201774, -0.1931382718666288, 0.18513157221738175, 0.3644950224471216, 0.6357152262965683, 0.0407551202152704, 0.08497422841423914, 0.3487012146468191, -0.08026315269369984, 0.24847819842610344, 0.15154242508973784, 0.14718998742413336, -0.21447080637046176, 0.12459898480373152, 0.7576997656965404, 0.25016582680075883, 0.09443410587745431, 0.23656590915416306, 0.13067120527321718, -0.2009705858236229, 0.20186887847241536, -0.3305009875026934, -0.03671781304236953, 0.18318971495928815, 0.07951070770111159, -0.21976908822922528, 0.295480608496454, -0.14395484916926138, -0.3052021083437517, -0.21667290341446233, -0.4315845363953309, -0.05338219720879526, -0.06413475303666524, 0.27059306781339815, 0.3096129080816439, 0.6633131691762129, 0.25816638223029886, -0.01067129402339933, 0.6825768944273405, -0.2737465782620136, -0.04536977002346369, 0.7442248768664693, 0.35760636471853746, -0.03790119244674228, -0.2420740079887751, -0.18475701439268913, 0.04828208745885672, -0.3008422504077641, 0.19306437500761695, 0.1115407432840864

In [22]:
glove_embedding['vectors'] = glove_embedding['vectors'].map(str)
glove_embedding['vectors'] = glove_embedding['key'] + glove_embedding['vectors']

In [23]:
for i in tqdm(range(len(glove_embedding))):
    glove_embedding['vectors'][i] = glove_embedding['vectors'][i].replace('[','').replace(']','').replace('\n','')

100%|██████████| 38890/38890 [00:07<00:00, 5394.15it/s]


In [24]:
glove_embedding.head()

,key,vectors
0,ㅇㅗㄹㄹㅣㅁ,ㅇㅗㄹㄹㅣㅁ-0.04014644 0.14591996 0.21770952 0.27505619 -0.21774962 -0.3299259 -0.34875664 0.15955163 -0.30599767 -0.09654434 -0.36265585 0.36642118 0.58957813 -0.25477741 -0.77548881 -0.16812614 -0.30386485 0.3014517 0.11541772 0.23872045 0.23607387 -0.32231933 0.65029271 -0.57035232 0.3518528 0.2721458 0.29503168 -0.24414353 0.24229787 -0.27020786 0.02250393 -0.64149403 -0.18691672 0.57878591 0.04031945 0.25008333 -0.11378724 -0.24900752 -0.36675543 -0.16945304 -0.26521683 0.47874288 -0.39373579 0.35531035 -0.0364649 0.24267091 0.28361149 0.0727594 -0.18305883 0.18589462 0.01891854 0.31339466 0.2110837 0.2616362 0.1850178 0.30698852 -0.34319437 -0.33101811 -0.0366438 -0.10888972 -0.33897783 -0.18377894 0.22498183 -0.16396334 0.17689501 -0.09917777 -0.23181384 -0.07816078 -0.32654061 0.39174611 0.09102456 0.05801771 -0.5810237 -0.15678637 0.12635572 0.00321863 0.06294066 -0.35182788 -0.04854145 -0.12647429 0.35109869 0.40514648 -0.21645254 -0.19755682 -0.32289444 -0.30027708 0.34837958 -0.04615759 0.35337252 -0.06437188 -0.30470855 0.79435022 0.25375599 -0.39471896 -0.04646276 0.28683884 0.56763656 -0.34304805 -0.13901353 -0.25175754 -0.36208369 0.05205259 -0.22300126 0.2905658 -0.36620431 -0.54267439 -0.55635134 -0.13487925 0.12809894 -0.27533946 -0.09402707 -0.13418316 0.31926451 0.25038203 0.56734689 0.50725485 0.13491343 -0.09576212 0.56955517 0.04188273 -0.26275488 0.27024855 -0.4486555 -0.37352145 -0.07066747 0.19620842 -0.05598441 -0.13595502 0.37554276 -0.24108017 -0.14587224 -0.02038195 -0.28231163 0.27694228 0.03949041 -0.36484638 0.25809373 0.03587891 -0.25194114 0.38679681 0.3846123 -0.36218281 -0.54193586 0.05941491 -0.22653054 -0.19972402 0.68535539 -0.20749228 -0.20579772 0.10440364 0.28986191 -0.41898958 -0.41396729 -0.21446917 -0.01976362 -0.33942792 0.55116389 0.35188357 0.22111976 0.29210891 -0.15658744 -0.19720351 0.36857953 0.45014699 -0.14041504 0.18623169 0.14295688 -0.2884932 -0.26274614 -0.16985756 -0.02450327 -0.59542114 -0.22030671 -0.15137482 0.10464277 -0.2418877 0.27974102 0.15631665 0.33281475 -0.11659845 -0.13164111 -0.41712243 0.2061083 0.58005565 -0.50544492 0.27397353 0.23080702 0.3383612 0.28207726 0.05866299 0.24901911 0.72161691 0.10578238 -0.03012658 -0.1668963 0.1184556 0.24818298 0.11677119 0.49216082 -0.1081244
1,ㅇㅡㄴㅎㅐㅇ,ㅇㅡㄴㅎㅐㅇ-0.1572792 -0.08054372 -0.27841671 0.21973004 0.13457085 -0.27744223 -0.40084188 0.30184945 -0.37914002 -0.26272903 -0.16365064 0.11286745 0.38496305 -0.33934461 -0.37365978 -0.29523862 -0.3034842 -0.08834723 -0.07348613 0.1021175 0.12998961 -0.30811428 -0.39247292 0.01906075 -0.07931723 0.05273994 0.6745661 -0.50186203 0.77431466 -0.23300256 -0.07766038 0.11865698 -0.09071159 0.1243992 -0.03220683 -0.39791699 -0.14834166 -0.32564888 0.32238752 -0.19313827 0.18513157 0.36449502 0.63571523 0.04075512 0.08497423 0.34870121 -0.08026315 0.2484782 0.15154243 0.14718999 -0.21447081 0.12459898 0.75769977 0.25016583 0.09443411 0.23656591 0.13067121 -0.20097059 0.20186888 -0.33050099 -0.03671781 0.18318971 0.07951071 -0.21976909 0.29548061 -0.14395485 -0.30520211 -0.2166729 -0.43158454 -0.0533822 -0.06413475 0.27059307 0.30961291 0.66331317 0.25816638 -0.01067129 0.68257689 -0.27374658 -0.04536977 0.74422488 0.35760636 -0.03790119 -0.24207401 -0.18475701 0.04828209 -0.30084225 0.19306438 0.11154074 0.20647254 0.53436873 -0.29766159 -0.2799169 0.28007286 -0.26512236 -0.24766068 -0.01053994 0.5284233 -0.31237856 -0.05439603 -0.33908246 -0.20242762 0.00664636 -0.13741498 0.03119037 -0.39012542 0.31199905 0.10927213 0.19657683 0.20031728 0.01557362 0.17524775 -0.10757874 0.18476346 0.25980472 0.09886934 0.30814531 -0.30672309 -0.00927756 -0.60975119 -0.14668808 0.03265322 0.19673261 -0.03530028 -0.11285591 0.14772563 0.27864426 -0.10311523 -0.21172293 -0.35814708 0.1042695 -0.03706134 0.19600204 -0.24872752 -0.23623717 -0.27154828 -0.53309937 0.11644491 -0.108796 -0.11860072 -0.12478401 0.17179854 0.0179788 0.06609469 0.03399539 -0.16891539 0.65338818 -0.0668

In [25]:
np.savetxt('../KB_NLP/glove.200D.100E.txt',glove_embedding['vectors'],fmt='%s')

---
## 1000 epochs, max loss

In [31]:
%%time
glove = Glove(no_components=200, learning_rate=0.01, random_state=42,max_loss=0.0002)
glove.fit(x.tocoo(), epochs=1000, no_threads=4, verbose=True)

Performing 1000 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Epoch 90
Epoch 91
Epoch 92
Epoch 93
Epoch 94
Epoch 95
Epoch 96
Epoch 97
Epoch 98
Epoch 99
Epoch 100
Epoch 101
Epoch 102
Epoch 103
Epoch 104
Epoch 105
Epo

---
# make every 10 th embeddings with 200dim

In [34]:
%%time
for k in range(10,210,10):
    glove = Glove(no_components=200, learning_rate=0.01, random_state=42,)
    glove.fit(x.tocoo(), epochs=k, no_threads=4, verbose=False)
    dictionary = {vocab:idx for idx, vocab in enumerate(idx2vocab)}
    glove.add_dictionary(dictionary)
    
    glove_embedding = pd.DataFrame()
    glove_embedding['key'] = list(glove.dictionary.keys())
    glove_embedding['vectors'] = list(glove.word_vectors)

    glove_embedding['vectors'] = glove_embedding['vectors'].map(str)
    glove_embedding['vectors'] = glove_embedding['key'] + glove_embedding['vectors']
    for i in tqdm(range(len(glove_embedding))):
        glove_embedding['vectors'][i] = glove_embedding['vectors'][i].replace('[','').replace(']','').replace('\n','')
    np.savetxt('../KB_NLP/glove_embedding/glove.200D.{}E.txt'.format(k),glove_embedding['vectors'],fmt='%s')
    print("{}th embedding DONE".format(k))

100%|██████████| 38890/38890 [00:07<00:00, 5170.59it/s]


10th embedding DONE


100%|██████████| 38890/38890 [00:07<00:00, 5178.05it/s]


20th embedding DONE


100%|██████████| 38890/38890 [00:07<00:00, 5293.16it/s]


...

180th embedding DONE


100%|██████████| 38890/38890 [00:07<00:00, 5276.32it/s]


190th embedding DONE


100%|██████████| 38890/38890 [00:07<00:00, 5478.10it/s]


200th embedding DONE
CPU times: user 1h 57min 17s, sys: 8.18 s, total: 1h 57min 26s
Wall time: 41min 18s


---
## 1000 epochs

In [16]:
%%time
glove = Glove(no_components=200, learning_rate=0.01, random_state=42,)
glove.fit(x.tocoo(), epochs=1000, no_threads=4, verbose=True)

Performing 100 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Epoch 90
Epoch 91
Epoch 92
Epoch 93
Epoch 94
Epoch 95
Epoch 96
Epoch 97
Epoch 98
Epoch 99
CPU times: user 5min 2s, sys: 47 ms, total: 5min 2s
Wall time: 1

In [22]:
dictionary = {vocab:idx for idx, vocab in enumerate(idx2vocab)}
glove.add_dictionary(dictionary)
print("shape: {}".format(glove.word_vectors.shape))

glove_embedding = pd.DataFrame()
glove_embedding['key'] = list(glove.dictionary.keys())
glove_embedding['vectors'] = list(glove.word_vectors)

glove_embedding['vectors'] = glove_embedding['vectors'].map(str)
glove_embedding['vectors'] = glove_embedding['key'] + glove_embedding['vectors']

In [23]:
for i in tqdm(range(len(glove_embedding))):
    glove_embedding['vectors'][i] = glove_embedding['vectors'][i].replace('[','').replace(']','').replace('\n','')

100%|██████████| 38890/38890 [00:07<00:00, 5394.15it/s]


In [24]:
glove_embedding.head()

,key,vectors
0,ㅇㅗㄹㄹㅣㅁ,ㅇㅗㄹㄹㅣㅁ-0.04014644 0.14591996 0.21770952 0.27505619 -0.21774962 -0.3299259 -0.34875664 0.15955163 -0.30599767 -0.09654434 -0.36265585 0.36642118 0.58957813 -0.25477741 -0.77548881 -0.16812614 -0.30386485 0.3014517 0.11541772 0.23872045 0.23607387 -0.32231933 0.65029271 -0.57035232 0.3518528 0.2721458 0.29503168 -0.24414353 0.24229787 -0.27020786 0.02250393 -0.64149403 -0.18691672 0.57878591 0.04031945 0.25008333 -0.11378724 -0.24900752 -0.36675543 -0.16945304 -0.26521683 0.47874288 -0.39373579 0.35531035 -0.0364649 0.24267091 0.28361149 0.0727594 -0.18305883 0.18589462 0.01891854 0.31339466 0.2110837 0.2616362 0.1850178 0.30698852 -0.34319437 -0.33101811 -0.0366438 -0.10888972 -0.33897783 -0.18377894 0.22498183 -0.16396334 0.17689501 -0.09917777 -0.23181384 -0.07816078 -0.32654061 0.39174611 0.09102456 0.05801771 -0.5810237 -0.15678637 0.12635572 0.00321863 0.06294066 -0.35182788 -0.04854145 -0.12647429 0.35109869 0.40514648 -0.21645254 -0.19755682 -0.32289444 -0.30027708 0.34837958 -0.04615759 0.35337252 -0.06437188 -0.30470855 0.79435022 0.25375599 -0.39471896 -0.04646276 0.28683884 0.56763656 -0.34304805 -0.13901353 -0.25175754 -0.36208369 0.05205259 -0.22300126 0.2905658 -0.36620431 -0.54267439 -0.55635134 -0.13487925 0.12809894 -0.27533946 -0.09402707 -0.13418316 0.31926451 0.25038203 0.56734689 0.50725485 0.13491343 -0.09576212 0.56955517 0.04188273 -0.26275488 0.27024855 -0.4486555 -0.37352145 -0.07066747 0.19620842 -0.05598441 -0.13595502 0.37554276 -0.24108017 -0.14587224 -0.02038195 -0.28231163 0.27694228 0.03949041 -0.36484638 0.25809373 0.03587891 -0.25194114 0.38679681 0.3846123 -0.36218281 -0.54193586 0.05941491 -0.22653054 -0.19972402 0.68535539 -0.20749228 -0.20579772 0.10440364 0.28986191 -0.41898958 -0.41396729 -0.21446917 -0.01976362 -0.33942792 0.55116389 0.35188357 0.22111976 0.29210891 -0.15658744 -0.19720351 0.36857953 0.45014699 -0.14041504 0.18623169 0.14295688 -0.2884932 -0.26274614 -0.16985756 -0.02450327 -0.59542114 -0.22030671 -0.15137482 0.10464277 -0.2418877 0.27974102 0.15631665 0.33281475 -0.11659845 -0.13164111 -0.41712243 0.2061083 0.58005565 -0.50544492 0.27397353 0.23080702 0.3383612 0.28207726 0.05866299 0.24901911 0.72161691 0.10578238 -0.03012658 -0.1668963 0.1184556 0.24818298 0.11677119 0.49216082 -0.1081244
1,ㅇㅡㄴㅎㅐㅇ,ㅇㅡㄴㅎㅐㅇ-0.1572792 -0.08054372 -0.27841671 0.21973004 0.13457085 -0.27744223 -0.40084188 0.30184945 -0.37914002 -0.26272903 -0.16365064 0.11286745 0.38496305 -0.33934461 -0.37365978 -0.29523862 -0.3034842 -0.08834723 -0.07348613 0.1021175 0.12998961 -0.30811428 -0.39247292 0.01906075 -0.07931723 0.05273994 0.6745661 -0.50186203 0.77431466 -0.23300256 -0.07766038 0.11865698 -0.09071159 0.1243992 -0.03220683 -0.39791699 -0.14834166 -0.32564888 0.32238752 -0.19313827 0.18513157 0.36449502 0.63571523 0.04075512 0.08497423 0.34870121 -0.08026315 0.2484782 0.15154243 0.14718999 -0.21447081 0.12459898 0.75769977 0.25016583 0.09443411 0.23656591 0.13067121 -0.20097059 0.20186888 -0.33050099 -0.03671781 0.18318971 0.07951071 -0.21976909 0.29548061 -0.14395485 -0.30520211 -0.2166729 -0.43158454 -0.0533822 -0.06413475 0.27059307 0.30961291 0.66331317 0.25816638 -0.01067129 0.68257689 -0.27374658 -0.04536977 0.74422488 0.35760636 -0.03790119 -0.24207401 -0.18475701 0.04828209 -0.30084225 0.19306438 0.11154074 0.20647254 0.53436873 -0.29766159 -0.2799169 0.28007286 -0.26512236 -0.24766068 -0.01053994 0.5284233 -0.31237856 -0.05439603 -0.33908246 -0.20242762 0.00664636 -0.13741498 0.03119037 -0.39012542 0.31199905 0.10927213 0.19657683 0.20031728 0.01557362 0.17524775 -0.10757874 0.18476346 0.25980472 0.09886934 0.30814531 -0.30672309 -0.00927756 -0.60975119 -0.14668808 0.03265322 0.19673261 -0.03530028 -0.11285591 0.14772563 0.27864426 -0.10311523 -0.21172293 -0.35814708 0.1042695 -0.03706134 0.19600204 -0.24872752 -0.23623717 -0.27154828 -0.53309937 0.11644491 -0.108796 -0.11860072 -0.12478401 0.17179854 0.0179788 0.06609469 0.03399539 -0.16891539 0.65338818 -0.0668

In [25]:
np.savetxt('../KB_NLP/glove.200D.1000E.txt',glove_embedding['vectors'],fmt='%s')